# Reddit Climate Change - Data Preparation
Supervision: Prof. Dr. Jan Fabian Ehmke

Group members: Britz Luis, Huber Anja, Krause Felix Elias, Preda Yvonne-Nadine

Time: Summer term 2023 

Data: https://www.kaggle.com/datasets/pavellexyr/the-reddit-climate-change-dataset

In [3]:
# Loading packages
import os
import pandas as pd

## Load data

In [4]:
# Loading data
raw_comments = pd.read_csv('data/the-reddit-climate-change-dataset-comments.csv', nrows=100)
# raw_posts = pd.read_csv('data/the-reddit-climate-change-dataset-posts.csv') # FIXME MAYBE ONLY USE COMMENTS?

## Pre-processing

In [ ]:
# Remove full duplicates
# create year/month/day/time column
# Keep only english comments
# use coherent columnnames
# Clean up data; filter out some artefacts (e.g. html tags, "\n" etc.) -> use regular expressions!

In [ ]:
# Feature engineering
raw_comments["post_id"] = [i.split("/")[6] for i in raw_comments.permalink]

## Save pre-processed data

In [ ]:
comment_save_path = f"data/preproc_comments.csv"

if not os.path.isfile(comment_save_path):
    raw_comments.to_csv(comment_save_path)
    print("File saved!")
else: 
    print("Warning file already exists")

In [ ]:
post_save_path = f"data/preproc_sample_posts.csv"

if not os.path.isfile(post_save_path):
    raw_posts.to_csv(post_save_path)
    print("File saved!")
else: 
    print("Warning file already exists")